In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import wfdb
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
"""
metadata_df = pd.read_csv('./metadata.csv')

records_data_mass = []

counter = 1
for index, file_metadata in tqdm(metadata_df.iterrows()):
    #print(str(counter)+'/'+str(metadata_df.shape[0]))
    counter = counter+1

    metadata_content = {
        'file': [],
        'fs': [],
        'duration_sec': [],
        'N_leads': [],
        'units': [],
        'sex': [],
        'Age': [],
        'Age group': [],
        'data': [],
        'lead_names': []}

    for key in list(metadata_content.keys()):
        if key not in ['data','lead_names']:
            metadata_content[key] = file_metadata[key]

    signals, fields = wfdb.rdsamp(file_metadata['file'])
    lead_names = fields['sig_name']

    metadata_content['data'] = signals
    metadata_content['lead_names'] = lead_names

    records_data_mass.append(metadata_content)

print(records_data_mass[0])
"""
#records_data_mass = pd.DataFrame(records_data_mass)
#records_data_mass['data'].apply(lambda x: x.shape)

"\nmetadata_df = pd.read_csv('./metadata.csv')\n\nrecords_data_mass = []\n\ncounter = 1\nfor index, file_metadata in tqdm(metadata_df.iterrows()):\n    #print(str(counter)+'/'+str(metadata_df.shape[0]))\n    counter = counter+1\n\n    metadata_content = {\n        'file': [],\n        'fs': [],\n        'duration_sec': [],\n        'N_leads': [],\n        'units': [],\n        'sex': [],\n        'Age': [],\n        'Age group': [],\n        'data': [],\n        'lead_names': []}\n\n    for key in list(metadata_content.keys()):\n        if key not in ['data','lead_names']:\n            metadata_content[key] = file_metadata[key]\n\n    signals, fields = wfdb.rdsamp(file_metadata['file'])\n    lead_names = fields['sig_name']\n\n    metadata_content['data'] = signals\n    metadata_content['lead_names'] = lead_names\n\n    records_data_mass.append(metadata_content)\n\nprint(records_data_mass[0])\n"

In [3]:
biodata_params = pd.read_csv("Bioage_data_params_table.csv", index_col=0)
biodata_params.dropna(subset=['age','age_group','sex'], inplace=True)
biodata_params.head()

,fname,age,age_group,sex,IMMEDIATE_CONTROL_OF_REGULATION,HEART_RATE,SDNN,RMSSD,STRESS_INDEX,TRIANGULAR_INDEX,...,R_S_RATIO_lead_V5,T_SYMMETRY_DERIVATIVES_lead_V1,T_SYMMETRY_AREAS_OF_TRIANGLES_lead_V1,QRS_T_AREA_RATIO_lead_V1,R_S_RATIO_lead_II,R_T_RATIO_lead_AVR,R_P_RATIO_lead_AVR,QRS_T_AREA_RATIO_lead_AVR,T_SYMMETRY_AREAS_OF_TRIANGLES_lead_AVR,T_SYMMETRY_DERIVATIVES_lead_AVR
0,A0002,49,40to49,Female,42.6,77.8,18.5,18.3,1056.0,36.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0016,14,1to17,Female,66.2,70.5,34.8,41.5,343.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A0020,27,18to29,Female,56.8,71.1,26.6,25.2,337.0,22.2,...,40.3000,0.500,1.300,0.348,NaN,NaN,NaN,NaN,NaN,NaN
3,A0029,35,30to39,Female,24.0,71.9,26.7,29.6,424.0,30.0,...,0.0308,0.333,0.333,4.000,0.069,NaN,NaN,NaN,NaN,NaN
4,A0030,46,40to49,Male,60.6,61.2,33.8,40.4,326.0,27.3,...,0.0714,NaN,NaN,NaN,36.600,NaN,NaN,NaN,NaN,NaN


In [4]:
biodata_params.drop(columns = ['HEART_BIO_AGE_FROM', "HEART_BIO_AGE_TO"], inplace=True)

In [5]:
(biodata_params.isna().sum().sort_values(ascending=False)/len(biodata_params))[:30]

T_SYMMETRY_DERIVATIVES_lead_AVR           0.999529
T_SYMMETRY_AREAS_OF_TRIANGLES_lead_AVR    0.999529
QRS_T_AREA_RATIO_lead_AVR                 0.999327
R_T_RATIO_lead_AVR                        0.976589
R_P_RATIO_lead_AVR                        0.933939
T_SYMMETRY_DERIVATIVES_lead_V1            0.605987
T_SYMMETRY_AREAS_OF_TRIANGLES_lead_V1     0.593878
QRS_T_AREA_RATIO_lead_V1                  0.581769
R_T_RATIO_lead_V1                         0.452069
FRACTAL_INDEX                             0.432291
T_SYMMETRY_AREAS_OF_TRIANGLES_lead_III    0.424420
T_SYMMETRY_DERIVATIVES_lead_III           0.419845
QRS_T_AREA_RATIO_lead_III                 0.416078
R_P_RATIO_lead_AVL                        0.325261
T_SYMMETRY_AREAS_OF_TRIANGLES_lead_AVL    0.318533
T_SYMMETRY_DERIVATIVES_lead_AVL           0.306626
R_S_RATIO_lead_II                         0.301985
QRS_T_AREA_RATIO_lead_AVL                 0.301783
R_S_RATIO_lead_AVF                        0.296401
R_T_RATIO_lead_III             

Encoding categorical features

In [6]:
group_to_num = {'1to17':0,'18to29':1, '30to39':2, '40to49':3, '50to59':4, '60to69':5, '70to89':6, '90+':7 }
sex_to_num = {"Female":0, "Male":1}
group_to_num, sex_to_num

({'1to17': 0,
  '18to29': 1,
  '30to39': 2,
  '40to49': 3,
  '50to59': 4,
  '60to69': 5,
  '70to89': 6,
  '90+': 7},
 {'Female': 0, 'Male': 1})

In [7]:
group_to_num_reversed = {value: key for key,value in group_to_num.items()}
sex_to_num_reversed = {value: key for key,value in sex_to_num.items()}
group_to_num_reversed, sex_to_num_reversed

({0: '1to17',
  1: '18to29',
  2: '30to39',
  3: '40to49',
  4: '50to59',
  5: '60to69',
  6: '70to89',
  7: '90+'},
 {0: 'Female', 1: 'Male'})

Drop cols with high nan percentage

In [8]:
nan_thresh = 0.2
drop_cols = biodata_params.columns[ (biodata_params.isna().sum().sort_values(ascending=False)/len(biodata_params))>= nan_thresh ].difference(['age',"age_group", "sex"])
drop_cols

Index(['ACTIVITY_OF_SUBCORTICAL_CENTERS', 'ACTIVITY_OF_VASOMOTOR_CENTERS',
       'CONDITION_OF_REGULATION_RESERVES', 'DFA', 'ENTROPY', 'FRACTAL_INDEX',
       'HEART_RATE', 'HF', 'IAE', 'IMMEDIATE_CONTROL_OF_REGULATION', 'LF',
       'LF_HF', 'LFn', 'PNN20', 'PNN50', 'RMSSD', 'SDNN', 'SDSD',
       'STRESS_INDEX', 'TOTAL_POWER', 'TRIANGULAR_INDEX', 'VLF', 'fname'],
      dtype='object')

In [9]:
biodata_params.drop(columns = drop_cols, inplace=True)
biodata_params.head()

,age,age_group,sex,HFn,INTEGRAL_INDICATOR_BY_IARS,IARS,STAMINA,ORGANIZATION_INDEX,MEAN,VHF,...,R_S_RATIO_lead_V5,T_SYMMETRY_DERIVATIVES_lead_V1,T_SYMMETRY_AREAS_OF_TRIANGLES_lead_V1,QRS_T_AREA_RATIO_lead_V1,R_S_RATIO_lead_II,R_T_RATIO_lead_AVR,R_P_RATIO_lead_AVR,QRS_T_AREA_RATIO_lead_AVR,T_SYMMETRY_AREAS_OF_TRIANGLES_lead_AVR,T_SYMMETRY_DERIVATIVES_lead_AVR
0,49,40to49,Female,94.8,43.0,7.0,45.5,0.862,771.0,30.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14,1to17,Female,83.5,100.0,3.0,67.8,0.760,851.0,52.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27,18to29,Female,52.2,100.0,3.0,64.3,0.751,844.0,16.4,...,40.3000,0.500,1.300,0.348,NaN,NaN,NaN,NaN,NaN,NaN
3,35,30to39,Female,95.7,68.0,5.0,50.7,0.767,834.0,65.9,...,0.0308,0.333,0.333,4.000,0.069,NaN,NaN,NaN,NaN,NaN
4,46,40to49,Male,92.1,68.0,5.0,60.8,0.736,980.0,19.2,...,0.0714,NaN,NaN,NaN,36.600,NaN,NaN,NaN,NaN,NaN


In [10]:
biodata_params.columns[ biodata_params.dtypes == 'object' ]

Index(['age_group', 'sex'], dtype='object')

Encoding categorical features

In [11]:
biodata_params['age_group'] = biodata_params['age_group'].apply(lambda x: group_to_num[x])
biodata_params['sex'] = biodata_params['sex'].apply(lambda x: sex_to_num[x])
biodata_params

,age,age_group,sex,HFn,INTEGRAL_INDICATOR_BY_IARS,IARS,STAMINA,ORGANIZATION_INDEX,MEAN,VHF,...,R_S_RATIO_lead_V5,T_SYMMETRY_DERIVATIVES_lead_V1,T_SYMMETRY_AREAS_OF_TRIANGLES_lead_V1,QRS_T_AREA_RATIO_lead_V1,R_S_RATIO_lead_II,R_T_RATIO_lead_AVR,R_P_RATIO_lead_AVR,QRS_T_AREA_RATIO_lead_AVR,T_SYMMETRY_AREAS_OF_TRIANGLES_lead_AVR,T_SYMMETRY_DERIVATIVES_lead_AVR
0,49,3,0,94.8,43.0,7.0,45.5,0.862,771.0,30.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14,0,0,83.5,100.0,3.0,67.8,0.760,851.0,52.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27,1,0,52.2,100.0,3.0,64.3,0.751,844.0,16.40,...,40.3000,0.500,1.300,0.348,NaN,NaN,NaN,NaN,NaN,NaN
3,35,2,0,95.7,68.0,5.0,50.7,0.767,834.0,65.90,...,0.0308,0.333,0.333,4.000,0.0690,NaN,NaN,NaN,NaN,NaN
4,46,3,1,92.1,68.0,5.0,60.8,0.736,980.0,19.20,...,0.0714,NaN,NaN,NaN,36.6000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14860,38,2,0,95.2,100.0,3.0,63.3,0.898,921.0,211.00,...,6.1400,0.667,1.500,1.000,0.0435,NaN,NaN,NaN,NaN,NaN
14861,43,3,1,93.9,100.0,3.0,46.9,0.846,817.0,1.85,...,0.0183,NaN,NaN,NaN,10.0000,NaN,NaN,NaN,NaN,NaN
14862,52,4,0,84.9,68.0,5.0,60.2,0.739,802.0,56.90,...,82.8000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14863,48,3,0,87.0,68.0,5.0,66.0,0.623,725.0,88.20,...,6.4300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Most correlate with target columns

In [12]:
biodata_params.drop(columns=['age', 'age_group']).corrwith(biodata_params['age']).sort_values(ascending=False, key = abs)[:50]

CV                                       -0.384776
ORGANIZATION_INDEX                        0.373212
J60_AMP_lead_I                           -0.286387
STAMINA                                  -0.286016
ST_END_AMP_lead_I                        -0.283781
J80_AMP_lead_I                           -0.283781
ST_END_AMP_lead_AVR                       0.275269
J80_AMP_lead_AVR                          0.275269
J40_AMP_lead_I                           -0.268205
ST_DISLOCATION_lead_I                    -0.268205
ST_MID_AMP_lead_I                        -0.268205
T_SYMMETRY_AREAS_OF_TRIANGLES_lead_AVR   -0.261014
J60_AMP_lead_AVR                          0.240498
FATIGUE                                   0.231772
QRS_T_AREA_RATIO_lead_I                   0.224622
J20_AMP_lead_I                           -0.216567
GLOBAL_QT_INT                             0.214692
R1_AMP_lead_III                          -0.205165
ST_END_AMP_lead_AVL                      -0.204830
J80_AMP_lead_AVL               

Drop columns with 1 unique values

In [13]:
drop_columns = biodata_params.columns[biodata_params.nunique()==1]
biodata_params.drop(columns = drop_columns, inplace = True)

In [14]:
biodata_params.nunique().sort_values(ascending=True)[:50]

sex                                        2
ALDRICH_CODE_INFERIOR                      2
T_SYMMETRY_DERIVATIVES_lead_AVR            3
HEART_RHYTHM_DISTURBANCES                  3
RISK_OF_HEART_DISORDERS                    3
MINNESOTA_CODE                             3
CONCLUSION                                 3
Q_SCORE_CODE                               4
I_LEAD_CODE                                4
SYNDROMIC_ECG_ANALYSIS                     4
OVERLOAD_OF_THE_RIGHT_VENTRICLE            6
ALDRICH_CODE_ANTERIOR                      6
T_SYMMETRY_AREAS_OF_TRIANGLES_lead_AVR     6
INTEGRAL_INDICATOR_BY_IARS                 6
OVERLOAD_OF_THE_LEFT_VENTRICLE             7
FROELICHER_CODE                            8
T_AREA_lead_AVR                            8
age_group                                  8
QRS_T_AREA_RATIO_lead_AVR                  9
IARS                                      10
VEB_PER_MINUTE                            11
SVEB_PER_MINUTE                           14
MODE_RR   

Impute missing values with mode

In [15]:
from sklearn.impute import SimpleImputer, KNNImputer
imputer = SimpleImputer(strategy = 'most_frequent')
biodata_params = pd.DataFrame(data = imputer.fit_transform(biodata_params),
                              columns = biodata_params.columns)
biodata_params

,age,age_group,sex,HFn,INTEGRAL_INDICATOR_BY_IARS,IARS,STAMINA,ORGANIZATION_INDEX,MEAN,VHF,...,R_S_RATIO_lead_V5,T_SYMMETRY_DERIVATIVES_lead_V1,T_SYMMETRY_AREAS_OF_TRIANGLES_lead_V1,QRS_T_AREA_RATIO_lead_V1,R_S_RATIO_lead_II,R_T_RATIO_lead_AVR,R_P_RATIO_lead_AVR,QRS_T_AREA_RATIO_lead_AVR,T_SYMMETRY_AREAS_OF_TRIANGLES_lead_AVR,T_SYMMETRY_DERIVATIVES_lead_AVR
0,49.0,3.0,0.0,94.8,43.0,7.0,45.5,0.862,771.0,30.30,...,0.0500,0.500,2.000,2.000,10.4000,0.333,0.0,1.25,1.0,0.5
1,14.0,0.0,0.0,83.5,100.0,3.0,67.8,0.760,851.0,52.30,...,0.0500,0.500,2.000,2.000,10.4000,0.333,0.0,1.25,1.0,0.5
2,27.0,1.0,0.0,52.2,100.0,3.0,64.3,0.751,844.0,16.40,...,40.3000,0.500,1.300,0.348,10.4000,0.333,0.0,1.25,1.0,0.5
3,35.0,2.0,0.0,95.7,68.0,5.0,50.7,0.767,834.0,65.90,...,0.0308,0.333,0.333,4.000,0.0690,0.333,0.0,1.25,1.0,0.5
4,46.0,3.0,1.0,92.1,68.0,5.0,60.8,0.736,980.0,19.20,...,0.0714,0.500,2.000,2.000,36.6000,0.333,0.0,1.25,1.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14860,38.0,2.0,0.0,95.2,100.0,3.0,63.3,0.898,921.0,211.00,...,6.1400,0.667,1.500,1.000,0.0435,0.333,0.0,1.25,1.0,0.5
14861,43.0,3.0,1.0,93.9,100.0,3.0,46.9,0.846,817.0,1.85,...,0.0183,0.500,2.000,2.000,10.0000,0.333,0.0,1.25,1.0,0.5
14862,52.0,4.0,0.0,84.9,68.0,5.0,60.2,0.739,802.0,56.90,...,82.8000,0.500,2.000,2.000,10.4000,0.333,0.0,1.25,1.0,0.5
14863,48.0,3.0,0.0,87.0,68.0,5.0,66.0,0.623,725.0,88.20,...,6.4300,0.500,2.000,2.000,10.4000,0.333,0.0,1.25,1.0,0.5


In [16]:
x_cols = biodata_params.columns.difference(['age', 'age_group'])
target = 'age'
#target = 'age_group'

Train-test-split

train - for traning model

test - to evaluate metrics

In [17]:
from sklearn.model_selection import train_test_split
train, test, _, _ = train_test_split(biodata_params, biodata_params[target], 
                                     stratify = biodata_params['age_group'],
                                    test_size = 0.1)

In [18]:
from sklearn.metrics import make_scorer, mean_absolute_error
scorer = make_scorer(mean_absolute_error, greater_is_better=False)

In [19]:
from sklearn.model_selection import cross_validate, KFold, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier


cv_results = cross_validate(
                            #RandomForestClassifier(n_estimators=200, n_jobs=-1, verbose=1),
                            RandomForestRegressor(n_estimators = 200, n_jobs=-1, verbose=1),
                           train[x_cols], train[target],
                           #cv = StratifiedKFold(n_splits=5, shuffle=True),
                            cv = 5,
                            return_estimator = True,
                           #scoring = 'accuracy'
                           scoring = 'neg_mean_absolute_error'
                            )

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  4.2min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   57.8s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  4.2min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | e

In [20]:
cv_results

{'fit_time': array([253.55844235, 251.25896883, 250.66360521, 253.11992216,
        254.60963821]),
 'score_time': array([0.12290788, 0.10904551, 0.11602902, 0.11204696, 0.12005973]),
 'estimator': [RandomForestRegressor(n_estimators=200, n_jobs=-1, verbose=1),
  RandomForestRegressor(n_estimators=200, n_jobs=-1, verbose=1),
  RandomForestRegressor(n_estimators=200, n_jobs=-1, verbose=1),
  RandomForestRegressor(n_estimators=200, n_jobs=-1, verbose=1),
  RandomForestRegressor(n_estimators=200, n_jobs=-1, verbose=1)],
 'test_score': array([-1.86962631, -2.15028774, -2.05610987, -2.01399065, -1.82368598])}

Quality report for each of 5 models

In [21]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae, r2_score as r2

for i, est in enumerate(cv_results['estimator']):
    y_pred = est.predict(test[x_cols])
    y_true = test[target]
    print(f"model{i}")
    #print(classification_report(y_true, y_pred))
    print(f"mean_squared_error: {mse(y_true, y_pred)}")
    print(f"mean_absolute_error: {mae(y_true, y_pred)}")
    print(f"r2_score: {r2(y_true, y_pred)}")
    

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s


model0
mean_squared_error: 11.254848083389374
mean_absolute_error: 1.806906523201076
r2_score: 0.9612035562053779
model1
mean_squared_error: 11.696203429724276
mean_absolute_error: 1.8376462676529925
r2_score: 0.9596821658000458
model2
mean_squared_error: 12.994641560188297
mean_absolute_error: 1.8775588433086752
r2_score: 0.955206336221885


[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.0s finished


model3
mean_squared_error: 11.971448621385338
mean_absolute_error: 1.8299663752521858
r2_score: 0.9587333715978587
model4
mean_squared_error: 11.942828631472763
mean_absolute_error: 1.814340954942838
r2_score: 0.9588320272013655


In [22]:
feats_names = x_cols

feats_importances = sum( [est.feature_importances_ for est in cv_results['estimator']] ) /len(cv_results['estimator'])

importance_df = pd.DataFrame()
importance_df['mean_importance'] = feats_importances
importance_df.set_index(feats_names, inplace=True)
importance_df.sort_values(by=['mean_importance'], ascending=False, inplace=True)

importance_df[:30]

,mean_importance
SDNNc,0.425383
CV,0.404771
MAX_HEART_RATE,0.052995
GLOBAL_RR_INT,0.014053
MEAN,0.007681
MIN_HEART_RATE,0.003709
GOLDEN_RATIO,0.003206
STAMINA,0.002499
J80_AMP_lead_I,0.001880
ST_END_AMP_lead_I,0.001850


Regression to classification

In [23]:
def convert_age_to_group(age):
    if age<18:
        return 0
    elif age>=18 and age<30:
        return 1
    elif age>=30 and age<40:
        return 2
    elif age>=40 and age<50:
        return 3
    elif age>=50 and age<60:
        return 4
    elif age>=60 and age<70:
        return 5
    elif age>=70 and age<90:
        return 6
    else:
        return 7

In [24]:
for i, est in enumerate(cv_results['estimator']):
    y_pred = [convert_age_to_group(pred) for pred in est.predict(test[x_cols]) ]
    y_true = test['age_group']
    print(f"model{i}")
    print("confusion matrix")
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s


model0
confusion matrix
[[ 22  11   0   0   0   0   0   0]
 [  2 136  15   1   0   0   0   0]
 [  0  31 141  13   0   0   0   0]
 [  0   0  22 241  13   3   0   0]
 [  0   0   0  22 310  11   0   0]
 [  0   0   0   2  17 264   6   0]
 [  0   0   1   0   2  37 163   0]
 [  0   0   0   0   0   0   1   0]]
              precision    recall  f1-score   support

         0.0       0.92      0.67      0.77        33
         1.0       0.76      0.88      0.82       154
         2.0       0.79      0.76      0.77       185
         3.0       0.86      0.86      0.86       279
         4.0       0.91      0.90      0.91       343
         5.0       0.84      0.91      0.87       289
         6.0       0.96      0.80      0.87       203
         7.0       0.00      0.00      0.00         1

    accuracy                           0.86      1487
   macro avg       0.75      0.72      0.74      1487
weighted avg       0.86      0.86      0.86      1487

model1
confusion matrix
[[ 23  10   0   0   

[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s


model2
confusion matrix
[[ 22  11   0   0   0   0   0   0]
 [  2 134  15   3   0   0   0   0]
 [  0  30 142  12   1   0   0   0]
 [  0   0  32 233  11   3   0   0]
 [  0   0   0  26 307  10   0   0]
 [  0   0   1   1  18 255  14   0]
 [  0   0   1   0   1  42 159   0]
 [  0   0   0   0   0   0   1   0]]
              precision    recall  f1-score   support

         0.0       0.92      0.67      0.77        33
         1.0       0.77      0.87      0.81       154
         2.0       0.74      0.77      0.76       185
         3.0       0.85      0.84      0.84       279
         4.0       0.91      0.90      0.90       343
         5.0       0.82      0.88      0.85       289
         6.0       0.91      0.78      0.84       203
         7.0       0.00      0.00      0.00         1

    accuracy                           0.84      1487
   macro avg       0.74      0.71      0.72      1487
weighted avg       0.85      0.84      0.84      1487

model3
confusion matrix
[[ 21  12   0   0   

[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.0s finished
